In [1]:
!pip install pytorch-lightning #==1.2.8 --quiet
!pip install transformers #==4.5.1 --quiet
!pip install tensorflow
!pip install keras-self-attention

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 585 kB 15.6 MB/s 
     |████████████████████████████████| 596 kB 46.2 MB/s 
     |████████████████████████████████| 419 kB 41.3 MB/s 
     |████████████████████████████████| 140 kB 71.5 MB/s 
     |████████████████████████████████| 1.1 MB 62.3 MB/s 
     |████████████████████████████████| 144 kB 68.8 MB/s 
     |████████████████████████████████| 94 kB 805 kB/s 
     |████████████████████████████████| 271 kB 7.6 MB/s 
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 12.9 MB/s 
     |████████████████████████████████| 86 kB 4.5 MB/s 
     |████████████████████████████████| 6.6 MB 40.5 MB/s 
Looking in indexes: 

In [2]:
#pytorch libraries 
import pytorch_lightning as pl
try:
    from torchmetrics.functional import accuracy, f1_score as f1, auroc
except ImportError:
    from pytorch_lightning.metrics.functional import accuracy, f1, auroc
#from pytorch_lightning.metrics.functional import accuracy, f1, auroc
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger
from tqdm.auto import tqdm
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import RobertaTokenizer, RobertaTokenizerFast , BertConfig, BertModel,RobertaModel, AdamW, get_linear_schedule_with_warmup, AutoTokenizer, AutoModel, AutoConfig
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, multilabel_confusion_matrix, roc_curve, auc, roc_auc_score, accuracy_score, plot_precision_recall_curve, precision_recall_curve

#Tensorflow libraries
#%tensorflow_version 2.x
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer, one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, Input, concatenate, GlobalMaxPool1D, Conv1D, LSTM, Dense, Dropout, Activation
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from itertools import cycle
from tensorflow.keras import backend as K
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Bidirectional, LSTM
from tensorflow.keras.layers import Input, ELU, Embedding, BatchNormalization, Convolution1D, MaxPooling1D, concatenate
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from keras_self_attention import SeqSelfAttention

import numpy as np
import pandas as pd
import re
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
import subprocess
import time
import datetime
import timeit

#import custom modules

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:

import sys
sys.path.insert(0,'/content/drive/MyDrive/Colab Notebooks/DGA_Version_70length/')
import tf_models, data_utils, utils
from init import pt_initialize, tf_initialize
from dga_transformer import DGADataset, DGADataModule, DGATransformer

#jupyter setup
%matplotlib inline
%config InlineBackend.figure_format='retina'
sns.set(style='whitegrid', palette='muted', font_scale=1.2)
HAPPY_COLORS_PALETTE = ["#01BEFE", "#FFDD00", "#FF7D00", "#FF006D", "#ADFF02", "#8F00FF"]
sns.set_palette(sns.color_palette(HAPPY_COLORS_PALETTE))
rcParams['figure.figsize'] = 12, 8

#initialization
RANDOM_SEED = 42
pl.seed_everything(RANDOM_SEED)
root = '/content/drive/MyDrive/Colab Notebooks/DGA_Version_70length/'
related_models = ['highnam.h5', 'lstm1_mi.h5', 'n-CDBC.h5', 'ENSEMBLE_CNN_BILSTM.h5']


Global seed set to 42
Global seed set to 42


# **Read dataset**

In [5]:
amrita = data_utils.read_amrita(f'{root}data/multi', 'tf')
train_df, val_df, test_df = data_utils.read_amrita(f'{root}data/multi')
umudga_df = data_utils.read_data(f'{root}data/multi/biumudga_test.csv')
unknown_df = data_utils.read_data(f'{root}data/multi/unknown_dga.csv')
charbot_df = data_utils.read_data(f'{root}data/multi/charbotsamples.csv')

#read the alexa top 1 million non dga websites
alexa = data_utils.read_data(f'{root}data/multi/alexa.csv')
#split alexa for training and testing
alexa_test = alexa[0:15000]
alexa_train = alexa[15000:]

# **Train and Save DGA_Transformer**

In [6]:
columns = train_df.columns.tolist()[:2] # 0 for DGA and 1 for benign
columns

['domain', 'benign']

In [7]:
train_df['benign'].value_counts()

0    241265
1     80934
Name: benign, dtype: int64

In [8]:
trans_train = pd.concat([
  train_df,
  alexa_train.sample(n=(len(train_df[train_df['benign'] == 0])-len(train_df[train_df['benign'] == 1])), random_state=42)
]).reset_index(drop = True)
trans_train = trans_train.sample(n=400_000, random_state=42)
trans_train.head()


,domain,benign
477202,phpsupport.ir,1
190629,qbwravmxwvcj.com,0
235389,nrylvaxvptz.info,0
452280,telemax.com.mx,1
40214,aqjjdfqugygeoo.com,0


In [ ]:
#val_df, val_details = data_utils.handle_imbalance(val_df, columns)
#test_df, test_details = data_utils.handle_imbalance(test_df, columns)

In [12]:
trans_train['benign'].value_counts(), val_df['benign'].value_counts(), test_df['benign'].value_counts()

(0    200031
 1    199969
 Name: benign, dtype: int64, 0    26884
 1     8916
 Name: benign, dtype: int64, 0    29628
 1    10150
 Name: benign, dtype: int64)

In [22]:
umudga_df['benign'].value_counts(), unknown_df['benign'].value_counts(), alexa_test['benign'].value_counts()

(0    15000
 1    10000
 Name: benign, dtype: int64, 0    15214
 Name: benign, dtype: int64, 1    15000
 Name: benign, dtype: int64)

In [21]:
24884+13916, (24884+13916)/400000, 29628+10150, (29628+10150)/400000, 39778*10

(38800, 0.097, 39778, 0.099445, 397780)

**Tokenization**

In [ ]:
tokenizer = RobertaTokenizerFast.from_pretrained(pt_initialize.MODEL_NAME, max_len=pt_initialize.MAX_LENGTH)

In [ ]:
data_module = DGADataModule(
  trans_train,
  val_df,
  test_df,
  tokenizer,
  train_batch_size=pt_initialize.TRAIN_BATCH_SIZE,
  val_batch_size= pt_initialize.EVAL_BATCH_SIZE,
  test_batch_size= pt_initialize.EVAL_BATCH_SIZE,
  max_token_len= pt_initialize.MAX_LENGTH, 
  LABELS = trans_train.columns.tolist()[1:]
)

**Training**

In [ ]:
#Optimizer scheduler
steps_per_epoch=len(trans_train) // pt_initialize.TRAIN_BATCH_SIZE
total_training_steps = steps_per_epoch * pt_initialize.MAX_EPOCH
print('steps_per_epoch: ', steps_per_epoch,'\ntotal_training_steps: ',total_training_steps)
#We'll use a fifth of the training steps for a warm-up:
warmup_steps = total_training_steps // 5
print('warmup_steps: ',warmup_steps, '\ntotal_training_steps: ', total_training_steps)

In [ ]:
trans_train.columns.tolist()[1:]

In [ ]:
model = DGATransformer(trans_train.columns.tolist()[1:],n_warmup_steps=warmup_steps,
                       n_training_steps=total_training_steps )

**Start Training**

In [ ]:
!rm -rf lightning_logs/
!rm -rf checkpoints/

In [ ]:
%load_ext tensorboard
%tensorboard --logdir ./lightning_logs

In [ ]:
checkpoint_callback = ModelCheckpoint(
  dirpath=f"{root}models/checkpoints",
  filename="bin_best-checkpoint",
  save_top_k=1,
  verbose=True,
  monitor="val_loss",
  mode="min"
)

In [ ]:
#Log the progress in TensorBoard:
logger = TensorBoardLogger(f"{root}logs/lightning_logs", name="domain_name")
#Early stopping triggers when the loss hasn't improved for the last 2 epochs 
#(you might want to remove/reconsider this when training on real-world projects)
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=2)

In [ ]:
from pytorch_lightning.plugins import DDPPlugin
trainer = pl.Trainer(
  logger=logger,
  checkpoint_callback=checkpoint_callback,
  callbacks=[early_stopping_callback],
  max_epochs=5, #pt_initialize.MAX_EPOCH,  
  gpus=pt_initialize.AVAIL_GPUS,
  #plugins=DDPPlugin(find_unused_parameters = False), 
  #precision = 16,
  accumulate_grad_batches=4,
  progress_bar_refresh_rate=30
)

#start training
trainer.fit(model, data_module)

In [ ]:
from transformers import RobertaConfig
RobertaConfig(pt_initialize.MODEL_NAME)

In [ ]:
torch.save(model.state_dict(), '/content/drive/MyDrive/Colab Notebooks/DGA_Version_70length/models/model_stats2epochalllayers.pt')

In [ ]:
torch.save(model, '/content/drive/MyDrive/Colab Notebooks/DGA_Version_70length/models/model_stats2epochalllayers.pt')

In [ ]:
trainer.checkpoint_callback.best_model_path

In [ ]:
trainer.save_checkpoint('/content/drive/MyDrive/Colab Notebooks/DGA_Version_70length/models/dgatrans4e4L.ckpt')

In [ ]:
.

# **Train and Save tf models**

In [ ]:

#prepare the character based tokenizer
tk, MAX_INDEX = tf_models.prepare_tokenizer(maxlen = tf_initialize.MAX_STRING_LENGTH)
X, y, class_list, classes = tf_models.text_2_sequence(amrita, tk, tf_initialize.MAX_STRING_LENGTH)

#split data into train, test, val
X_train, X_test_, y_train, y_test_ = train_test_split(X, y, test_size=0.20, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_test_, y_test_, test_size=0.50, random_state=42)


In [ ]:
LABEL_COLUMNS = amrita.columns.tolist()[:2] # 0 for DGA and 1 for benign
LABEL_COLUMNS

In [ ]:
def get_conv_layer(emb, kernel_size=5, filters=256):
    # Conv layer
    conv = Convolution1D(kernel_size=kernel_size, filters=filters)(emb)
    conv = ELU()(conv)
    conv = MaxPooling1D(5)(conv)
    conv = Dropout(0.5)(conv)
    return conv

def cnn_bilstm_att(max_len=256, emb_dim=32, max_vocab_len=39, W_reg=regularizers.l2(1e-4), LABELS_NO = 21):
    # Input
    main_input = Input(shape=(max_len,), dtype='int32', name='main_input')

    # Embedding layer
    emb = Embedding(input_dim=max_vocab_len, output_dim=emb_dim, input_length=max_len)(main_input)
    emb = Dropout(0.2)(emb)

    conv2 = get_conv_layer(emb, kernel_size=2, filters=256)
    conv3 = get_conv_layer(emb, kernel_size=3, filters=256)
    conv4 = get_conv_layer(emb, kernel_size=4, filters=256)
    conv5 = get_conv_layer(emb, kernel_size=5, filters=256)

    bilstm = Bidirectional(LSTM(units=128, return_sequences=True, dropout=0.2))(emb)

    att = SeqSelfAttention(attention_activation='relu')(bilstm)

    cnnlstm_merged = concatenate([conv2, conv3, conv4, conv5, att], axis=1)
    cnnlstm_merged = Flatten()(cnnlstm_merged)

    hidden1 = Dense(4128)(cnnlstm_merged)
    hidden1 = ELU()(hidden1)
    hidden1 = BatchNormalization()(hidden1)
    hidden1 = Dropout(0.5)(hidden1)

    hidden2 = Dense(1024)(hidden1)
    hidden2 = ELU()(hidden2)
    hidden2 = BatchNormalization()(hidden2)
    hidden2 = Dropout(0.5)(hidden2)

    # Output layer (last fully connected layer)
    output = Dense(LABELS_NO, activation='softmax', name='output')(hidden2)

    # Compile model and define optimizer
    model = Model(inputs=[main_input], outputs=[output])
    
    adam = Adam(lr=1e-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
    model.compile(optimizer=adam, loss='categorical_crossentropy',metrics=['accuracy', tf.keras.metrics.CategoricalAccuracy()])
    
    return model

In [ ]:
tf_initialize.MAX_INDEX, tf_initialize.MAX_STRING_LENGTH

In [ ]:
del model

In [ ]:
model = tf_models.build_LSTM_MI_model(tf_initialize.MAX_INDEX, 128)

In [ ]:
model.summary()

In [ ]:
model.fit(X_train,y_train,validation_data=(X_val,y_val),epochs=5,batch_size=64)

In [ ]:
related_models = ["ENSEMBLE_CNN_BILSTM"]
device_name = utils.get_tf_device()
for model_name in related_models:  #['highnam.h5', 'lstm1_mi.h5', 'n-CDBC.h5']
  total_stats = {}
  with tf.device(device_name):
    if model_name == 'highnam.h5':
      model = tf_models.build_highnam()
    elif model_name == 'lstm1_mi.h5':
      model = tf_models.build_LSTM_MI_model(tf_initialize.MAX_INDEX, tf_initialize.MAX_STRING_LENGTH)
    elif model_name == "ENSEMBLE_CNN_BILSTM":
      model = cnn_bilstm_att(LABELS_NO=1)
      mc = ModelCheckpoint(filepath='./trained_models/' + model_name+ '.hdf5', monitor='val_loss', mode='min',save_best_only=True, verbose=1)
    else:
      model = tf_models.build_CBDC()
    t0 = time.time() 
    if model_name == "ENSEMBLE_CNN_BILSTM":
       model.fit(X_train,y_train,validation_data=(X_val,y_val),epochs=5,batch_size=64, callbacks=[mc])
       model.save_weights(f'{root}models/{model_name}')
    else:
      model.fit(X_train,y_train,validation_data=(X_val,y_val),epochs=5,batch_size=64)
      model.save(f'{root}models/{model_name}')
    t1 = utils.format_time(time.time() - t0)
    print(f'{model_name} training time: ', t1)
    

# **Prepare Related work models**

In [ ]:
.

In [ ]:
LABEL_COLUMNS = amrita.columns.tolist()[:2] # 0 for DGA and 1 for benign
LABEL_COLUMNS

In [ ]:
#store testing data
training_stats = []
testing_record = []
THRESHOLD, upper, lower = 0.5, 1 , 0 
device_name = utils.get_tf_device()
test_time = time.time() 

def add_stats(model_name, dataset, acc, prec, fscore, recall, test_time, DGAs, benign, aucurve):
  # Record all statistics from this epoch.
  training_stats.append(
      {
          'Model_Name':model_name,
          'Dataset': dataset,
          'Test Time': test_time,
          'Accuracy': acc,
          'Precision': prec,
          'Recall': recall,
          'F Score': fscore,
          'AUROC': aucurve,
          'DGA Samples':DGAs,
          'Benign Samples': benign
      }
  )


def test_model(model, X_test):
  t0 = time.time() 
  predictions = model.predict(X_test, verbose=True)
  test_time = utils.format_time(time.time() - t0)
  #yhat = model.predict_classes(X_test)
  #yhat = model.predict_proba(X_test)  
  return predictions, test_time

#load and test the

#prepare the character based tokenizer
tk, MAX_INDEX = tf_models.prepare_tokenizer(maxlen = tf_initialize.MAX_STRING_LENGTH)
X, y, class_list, classes = tf_models.text_2_sequence(amrita, tk, tf_initialize.MAX_STRING_LENGTH)

#split data into train, test, val
X_train, X_test_, y_train, y_test_ = train_test_split(X, y, test_size=0.20, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_test_, y_test_, test_size=0.50, random_state=42)

#start testing
for model_name in related_models:  #['highnam.h5', 'lstm1_mi.h5', 'n-CDBC.h5']
  total_stats = {}
  with tf.device(device_name):
    if model_name == 'highnam.h5':
      model = tf_models.build_highnam()
    elif model_name == 'lstm1_mi.h5':
      model = tf_models.build_LSTM_MI_model()
    else:
      model = tf_models.build_CBDC()

    #load the pretrained model    
    trained_model =  tf.keras.models.load_model(f'{root}models/{model_name}')
    predictions, test_time = test_model(model, X_test)
    y_pred = np.where(predictions > THRESHOLD, upper, lower)
    
    #Confusion Matrix
    confusion_matrix = confusion_matrix(y_test, y_pred)

    #Accuracy
    acc = accuracy_score(y_test, y_pred)

    #Compute AUROC
    print("AUROC per tag")
    aurc_val = []
    for i, name in enumerate([LABEL_COLUMNS[1:]]):
      tag_auroc = roc_auc_score(y_test, y_pred)
      print(f"{name}: {tag_auroc}")
      aurc_val.append(tag_auroc)

    #Compute  classification report 
    class_rpt = classification_report(y_test, y_pred, digits=4)
    total_stats['Model'] = model_name
    total_stats['y_pred'] = y_pred
    total_stats['y_test'] = y_test
    total_stats['confusion_matrix'] = confusion_matrix
    total_stats['acc'] = acc
    total_stats['classification_report'] = class_rpt
    total_stats['auroc'] = aurc_val
    total_stats['test_time'] = test_time
    testing_record.append(total_stats)





  

  
